# Deconstructed Interactive Demo

This is a deconstructed version of the demo that is intended to show a bit more detail about the operation of `prospector`. It runs over the same example data as in the `demo_params.py` parameter file, using a similar model.  However, in this demo the implicit methods of the parameter file are made explicit.

First let's set up some environmental dependencies. These just make the numerics easier and adjust some of the plotting defaults to make things more legible.

In [1]:
%matplotlib inline
import time, sys, os
import h5py
import numpy as np
from matplotlib.pyplot import *
# re-defining plotting defaults
from matplotlib.font_manager import FontProperties
from matplotlib import gridspec
from prospect.utils.obsutils import fix_obs
import fsps
import sedpy
import prospect
from prospect.models import priors
from prospect.models.templates import TemplateLibrary
rcParams.update({'xtick.major.pad': '7.0'})
rcParams.update({'xtick.major.size': '7.5'})
rcParams.update({'xtick.major.width': '1.5'})
rcParams.update({'xtick.minor.pad': '7.0'})
rcParams.update({'xtick.minor.size': '3.5'})
rcParams.update({'xtick.minor.width': '1.0'})
rcParams.update({'ytick.major.pad': '7.0'})
rcParams.update({'ytick.major.size': '7.5'})
rcParams.update({'ytick.major.width': '1.5'})
rcParams.update({'ytick.minor.pad': '7.0'})
rcParams.update({'ytick.minor.size': '3.5'})
rcParams.update({'ytick.minor.width': '1.0'})
rcParams.update({'xtick.color': 'k'})
rcParams.update({'ytick.color': 'k'})
rcParams.update({'font.size': 30})

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1069: UserWarning: Bad val "$TEMPLATE_BACKEND" on line #41
	"backend      : $TEMPLATE_BACKEND
"
	in file "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle"
	Key backend: Unrecognized backend string "$template_backend": valid strings are [u'pgf', u'cairo', u'MacOSX', u'CocoaAgg', u'gdk', u'ps', u'GTKAgg', u'nbAgg', u'GTK', u'Qt5Agg', u'template', u'emf', u'GTK3Cairo', u'GTK3Agg', u'WX', u'Qt4Agg', u'TkAgg', u'agg', u'svg', u'GTKCairo', u'WXAgg', u'WebAgg', u'pdf']
  (val, error_details, msg))
/Applications/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Setup

We need several things to run a fit.  These include 
  1. An `obs` dictionary (with the data we intend to fit)
  2. A stellar population synthesis object (to predict spectra from parameters)
  3. A `model` object (to store and translate parameters and priors)
  4. A likelihood function
  
It can also useful to collect the meta-parameters contolling how the fit is done in a ``run_params`` dictionary. We will do that as we go along.  We will also encapsulate each step of the setup in a series of `load_x()` functions.

### The `load_obs` function 
It can be helpful to encapsulate all the logic for reading from your catalog into a single `load_obs` method.  Let's do that now.

In [2]:
def load_obs(snr=10, ldist=10.0, **extras):
    galex = ['galex_FUV', 'galex_NUV']
    spitzer = ['spitzer_irac_ch'+n for n in ['1','2','3','4']]
    sdss = ['sdss_{0}0'.format(b) for b in ['u','g','r','i','z']]
    filternames = galex + sdss + spitzer
    
    # Here you could, e.g. read from a catalog
    M_AB = np.array([-11.93, -12.37, -13.37, -14.22, -14.61, -14.86, 
                     -14.94, -14.09, -13.62, -13.23, -12.78])
    dm = 25 + 5.0 * np.log10(ldist)
    mags = M_AB + dm

    obs = {}
    obs["filters"] = sedpy.observate.load_filters(filternames)
    obs["maggies"] = 10**(-0.4*mags)
    obs["maggies_unc"] = (1./snr) * obs["maggies"]
    obs["phot_mask"] = np.array(['spitzer' not in f.name for f in obs["filters"]])
    obs["phot_wave"] = [f.wave_effective for f in obs["filters"]]
    obs["wavelength"] = None  # this would be a vector of wavelengths in angstroms if we had 
    obs["spectrum"] = None
    obs['unc'] = None  #spectral uncertainties are given here
    obs['mask'] = None

    return obs

And we will store some meta-parameters that control the input arguments to this method:

In [3]:
run_params = {}
run_params["snr"] = 10.0
run_params["ldist"] = 10.0
obs=load_obs(**run_params)
obs = fix_obs(obs)
print(obs.keys())
print obs['ndof']
print obs['filternames']

['phot_mask', 'phot_wave', 'ndof', 'mask', 'spectrum', 'maggies_unc', 'filternames', 'unc', 'maggies', 'filters', 'wavelength', 'logify_spectrum']
7
['galex_FUV', 'galex_NUV', 'sdss_u0', 'sdss_g0', 'sdss_r0', 'sdss_i0', 'sdss_z0', 'spitzer_irac_ch1', 'spitzer_irac_ch2', 'spitzer_irac_ch3', 'spitzer_irac_ch4']


## The model object

Now we need a set of model parameters, which will **define** the model we are tying to fit to the data.  The model object stores the parameters that are used by the SPS object to build a spectrum, as well as infomation about which parameters are to be varied during fitting, and priors on those parameters.  It efficiently converts between a vector of parameter values (the `theta` attribute) used by the MCMC samplers or optimizers and the dictionary of parameter names and values (the `params` attribute) that can be passed to the sps objects' `get_spectrum()` method.

To create the model object we need a list or dictionary of model parameters and some infomation about them.  Each parameter must a have a name, a length (vector parameters are possible), an initial value, and must be specified as either a free parameter or a fixed parameter.  If it is a free parameter it needs a prior as well, which we will get from the `priors` module.

### Building by hand
Let's start by creating a dictionary that describes a single parameter controlling the stellar mass.

Ok, that's the description for one parameter.  Let's **build up the rest of our model** as a dictionary of these parameter descriptions.  At a minimum we will need some sort of distance or redshift information (which in this example is a fixed parameter), and something descibing the SFH.  We could also add parameters controlling metallicity, dust attenuation and emission, nebular emission, even the IMF. Note that any parameter whose value is not explicitly specified via a model parameter dictionary will be given the default value from python-FSPS.

The easiest way to build up this model is to start with some predefined parameter sets from the `prospect.models.templates` module. First, lets look at what pre-packaged parameter sets are available.

It looks like the `"parametric_sfh"` parameter set will do most of what we want.  Let's look at it in more detail.

In [5]:
#TemplateLibrary.describe("parametric_sfh")
#TemplateLibrary.describe("agn")

print '###############################'
TemplateLibrary.describe("alpha")


###############################
Free Parameters: (name: prior) 
-----------
  dust1: <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=4.0)
  duste_qpah: <class 'prospect.models.priors.TopHat'>(mini=0.5,maxi=7.0)
  total_mass: <class 'prospect.models.priors.LogUniform'>(mini=100000000.0,maxi=1e+12)
  dust_index: <class 'prospect.models.priors.TopHat'>(mini=-2.0,maxi=0.5)
  duste_gamma: <class 'prospect.models.priors.LogUniform'>(mini=0.001,maxi=0.15)
  agn_tau: <class 'prospect.models.priors.LogUniform'>(mini=5.0,maxi=150.0)
  logzsol: <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)
  dust2: <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=4.0)
  fagn: <class 'prospect.models.priors.LogUniform'>(mini=1e-05,maxi=3.0)
  z_fraction: <class 'prospect.models.priors.Beta'>(mini=0.0,beta=[1 1 1 1 1],maxi=1.0,alpha=[5 4 3 2 1])
  duste_umin: <class 'prospect.models.priors.TopHat'>(mini=0.1,maxi=25)

Fixed Parameters: (name: value [, depends_on]) 
-----------
  add_dust_emis

That looks pretty good.  We'll just adjust a couple of the initial values and priors, and add a parameter that will set the distance even though the redshift of theses objects is zero.

### Manipulating the `model` object
Great.  We now have a model.  We're fitting for 5 parameters here:
- stellar mass *formed* $M_\star$,
- metallicity $\log Z/Z_\odot$, 
- age $t$ of the galaxy 
- star formation timescale $\tau$ for an exponentially declining star formation history (SFH), and
- dust attenuation of old stellar populations $A_V$.

Everything else here is fixed explicitly.  There are many other `sps` parameters that are set implicitly in the FSPS defaults.  Note that by default the stellar mass here refers to the stellar mass *formed* by the given age, which will always be slightly higher than the *surviving* stellar mass, due to mass loss during stellar evolution (winds, SNe, etc.)

Let's change some of the parameter values in a couple different ways, and look at the prior probablity for those new parameter combinations.

### The `load_model` function
As for the ``obs`` dictionary and the ``sps`` object it can be sueful to define a ``load_model`` method that instantiates the ``model`` object.  This method can also be used to change the parameter specification dictionaries according to adjustable arguments.  Let's do an example where we change whether the metallicity is a free and fitted parameter or fixed to a particular value, optionally turn on dust emission, and where we can set the redshift by hand as well.

In [6]:
def load_model(object_redshift=None, fixed_metallicity=None, add_dust=False, 
               ldist=10.0, **extras):
    
    from prospect.models.sedmodel import SedModel
    from prospect.models.templates import TemplateLibrary

    model_params = TemplateLibrary["alpha"]
    
    # Add lumdist parameter
    model_params["lumdist"] = {"N": 1, "isfree": False, "init": ldist, "units":"Mpc"}
    
    model_params["zred"]['init'] = object_redshift
    # adjust priors
    model_params["duste_qpah"]["isfree"] = True
    model_params["duste_umin"]["isfree"] = True
    model_params["duste_gamma"]["isfree"] = True
    #model_params["dust1"]["isfree"] = True
    
    model_params["zred"]["init"] = 0.0
    model_params["dust2"]["init"] = 0.3
    model_params["logzsol"]["init"] = -0.5
    model_params["total_mass"]["init"] = 1e8
    
    #model_params['z_fraction']['prior'] = priors.Beta(alpha=1.0,beta=1.0,mini=0.0,maxi=1.0)
    model_params["dust_index"]["prior"] = priors.TopHat(mini=-2.2, maxi=0.4)
    model_params["logzsol"]["prior"] = priors.TopHat(mini=-2.0, maxi=0.2)
    model_params["duste_qpah"]["prior"] = priors.TopHat(mini=0.1, maxi=10)
    model_params["duste_umin"]["prior"] = priors.TopHat(mini=0.1, maxi=25)
    model_params["duste_gamma"]["prior"] = priors.TopHat(mini=1e-6, maxi=1.0)
    model_params["dust2"]["prior"] = priors.TopHat(mini=1e-6, maxi=3.0)
    model_params["agn_tau"]["prior"] = priors.LogUniform(mini=5, maxi=1.5e2)
    model_params["total_mass"]["prior"] = priors.LogUniform(mini=1e5, maxi=1e14)
    # If we are going to be using emcee, it is useful to provide a 
    # minimum scale for the cloud of walkers (the default is 0.1)
    model_params["total_mass"]["disp_floor"] = 1e5
    model_params["agn_tau"]["disp_floor"] = 1
    model_params["duste_gamma"]["disp_floor"] = 0.1

    
    # Change the model parameter specifications based on some keyword arguments

    if object_redshift is not None:
        # make sure zred is fixed
        model_params["zred"]['isfree'] = False
        # And set the value to the object_redshift keyword
        model_params["zred"]['init'] = object_redshift
        

        
    # Now instantiate the model using this new dictionary of parameter specifications
    model = SedModel(model_params)

    return model

run_params["object_redshift"] = 0.0


model = load_model(**run_params)
#print model.fixed_params
print '---------'
print model.free_params

#print model.init_config

---------
['total_mass', 'dust_index', 'dust1', 'logzsol', 'dust2', 'duste_qpah', 'z_fraction', 'duste_umin', 'agn_tau', 'duste_gamma', 'fagn']


## The `sps` object
We need an object that will build SEDs for a given set of parameters.  In `prospector` we call these **sps** objects.  Given a dictionary of parameters (provided by the model object), they must be able to return a spectrum, photometry, and maybe some ancillary information.  This is often done using large spectral libraries and, for galaxies, isochrone information.  Typically in `prospector` we use `fsps.StellarPopulation` objects, under thin wrappers that add a little functionality and change the API a bit. The different wrappers correspond to different SFH parameterizations. Here we use `CSPSpecBasis` which works with (linear combinations of) composite stellar populations as described in the FSPS manual with `sfh_type` of 1, 4, or 5.  Other `sps` objects can be used for non-parameteric SFH, notable `prospect.sources.FastStepBasis`.

### The `load_sps` function
Again, it can be helpful to encapsulate the loading of the sps object in a `load_sps` method, with meta-parameters controlling the how the object is instantiated

In [7]:
def load_sps(zcontinuous=1, **extras):
    from prospect.sources import FastStepBasis
    sps = FastStepBasis(zcontinuous=zcontinuous)
    return sps

run_params["zcontinuous"] = 1
sps = load_sps(**run_params)


### View the model
Now that we have the `sps` object we can also generate a prediction for the data from any set of model parameters.  To see how this works, lets make an SED!

Note that creating a new model with FSPS is somewhat time-intensive (of order seconds), but once the relevant SSPs have been built they are subsequently stored in cache so similar models can be generated much more quickly (of order milliseconds, unless you are changing parameters that affect the SSPs, like the IMF)

## Likelihood function
Now all we are missing is a likelihood function.  In most cases, this will simply be a function of the **spectral likelihood** and a **photometric likelihood** such that

$$
\mathcal{L} = f(\mathcal{L}_{\textrm{spec}}, \mathcal{L}_{\textrm{phot}}) \quad .
$$

Assuming our errors are Normal (i.e. Gaussian), the log-likelihoods for each component are extremely straightforward to define and can be imported directly from Prospector.  How we choose to combine these likelihoods might vary depending on the particulars of our data. For the demo, our likelihood function for our model parameters $\boldsymbol{\theta}$ is just

$$
\ln\mathcal{L}(\boldsymbol{\theta}) = \ln\mathcal{L}_{\textrm{spec}}(\boldsymbol{\theta}) + \ln\mathcal{L}_{\textrm{phot}}(\boldsymbol{\theta}) \quad .
$$

Below is a simple version of the likelihood function used in `prospector`.  Note that more complicated likelihoods including covariant noise and fitted noise parameters are possible, using special NoiseModel classes within `prospector`.  Also we are using the globally defined `obs`, `model`, and `sps`; in principle these could be passed as arguments to the `lnprobfn`.

For nested sampling `lnprobfn(theta, nested=True)` will return the likelihood (since the prior probability is accounted for by drawing proposals from the priors), while for other types of MCMC sampling `lnprobfn(theta, nested=False)` returns the posterior probability.

When using the provided scripts (`prospector.py` and `prospector_nest.py`) you don't have to specify the likelihood function yourself, it is defined in those scripts.

In [8]:
from prospect.likelihood import lnlike_spec, lnlike_phot, write_log
from prospect.models import priors
verbose = False
def lnprobfn(theta, nested=False, verbose=verbose):
    """
    Given a parameter vector, a dictionary of observational data 
    a model object, and an sps object, return the ln of the posterior. 
    This requires that an sps object (and if using spectra 
    and gaussian processes, a GP object) be instantiated.
    """

    # Calculate prior probability and exit if not within prior
    # Also if doing nested sampling, do not include the basic priors, 
    # since the drawing method includes the prior probability
    lnp_prior = model.prior_product(theta, nested=nested)
    if not np.isfinite(lnp_prior):
        return -np.infty
        
    # Generate "mean" model
    t1 = time.time()
    spec, phot, mfrac = model.mean_model(theta, obs, sps=sps)
    d1 = time.time() - t1
 
    # Calculate likelihoods
    t2 = time.time()
    lnp_spec = lnlike_spec(spec, obs=obs)
    lnp_phot = lnlike_phot(phot, obs=obs)
    d2 = time.time() - t2
    if verbose:
        write_log(theta, lnp_prior, lnp_spec, lnp_phot, d1, d2)

    return lnp_prior + lnp_phot + lnp_spec

run_params["verbose"] = verbose

It can also be useful for some optimization methods (i.e. Levenberg-Marquardt) to define a function that returns the vector of chi-square residuals.  Let's do that.

In [9]:
from prospect.likelihood import chi_spec, chi_phot
def chivecfn(theta):
    """A version of lnprobfn that returns the simple uncertainty 
    normalized residual instead of the log-posterior, for use with 
    least-squares optimization methods like Levenburg-Marquardt.
    
    It's important to note that the returned chi vector does not 
    include the prior probability.
    """
    lnp_prior = model.prior_product(theta)
    if not np.isfinite(lnp_prior):
        return -np.infty

    # Generate mean model
    t1 = time.time()
    try:
        spec, phot, x = model.mean_model(theta, obs, sps=sps)
    except(ValueError):
        return -np.infty
    d1 = time.time() - t1

    chispec = chi_spec(spec, obs)
    chiphot = chi_phot(phot, obs)
    return np.concatenate([chispec, chiphot])

# Running Prospector
Now that we have defined the model and set up the data that we want to fit, we are ready to run prospector.  We will do this in a few steps.  First we will run all the convenience functions we made earlier to get the fitting ingredients and set up the output.  Then we will conduct a $\chi^2$ minimization. Finally we will run an ensemble MCMC sampler around the best location form the minimization and save the output.

## Preparing
Here we will run all our convenience functions and set up the output file(s) where our results will be saved.

### Set Up Outputs
We are just about ready to do the inference. But before we start running anything, let's just check whether we can write out our fits to an HDF5 file. If not, we'll just dump them as a pickle at the end.  As the HDF5 output is set up, the `model_params` list, the `obs` dictionary, and the `run_params` dictionary are serialized (if possible) and added as attributes or datasets in the HDF5 file.

In [10]:
from prospect.io import write_results

In [11]:
run_params["outfile"] = 'demo'
outroot = "{0}_{1}".format(run_params['outfile'], int(time.time()))
try:
    hfilename = outroot + '_mcmc.h5'
    hfile = h5py.File(hfilename, "a")
    print("Writing to file {}".format(hfilename))
    write_results.write_h5_header(hfile, run_params, model)
    write_results.write_obs_to_h5(hfile, obs)
except:
    hfile = None

Writing to file demo_1560288828_mcmc.h5
Could not serialize model_params


## Minimization Step
We can attempt to initialize our model reasonably close to the data by using some numerical minimization routines.
Here we will use Levenberg-Marquardt. Keywords that control the optimization algorithm will again be stored in the `run_params` dictionary. Levenberg-Marquardt requires a likelihood function that returns a vector of chi values, not an actual likelihood, so we will write that. It's important to note that in the function below we do *not* account for prior probabilities.

In [12]:
from prospect import fitting
from scipy.optimize import least_squares
run_params["nmin"] = 5
run_params['ftol'] = 3e-16 
run_params['maxfev'] = 5000
run_params['xtol'] = 3e-16

In [13]:
# --- start minimization ----
# find the best initial value
min_method = 'levenberg_marquardt'
run_params["min_method"] = min_method

# We'll start minimization from "nmin" separate places, 
# the first based on the "init" values of each parameter and the 
# rest drawn from the prior.  This can guard against local minima.
nmin = run_params["nmin"]
ts = time.time()  # time it
pinitial = fitting.minimizer_ball(model.initial_theta.copy(), nmin, model)
print model.initial_theta.copy() 
guesses = []
for i, pinit in enumerate(pinitial): #loop over initial guesses
    res = least_squares(chivecfn, np.array(pinit), method='lm', x_scale='jac',
                        xtol=run_params["xtol"], ftol=run_params["ftol"], 
                        max_nfev=run_params["maxfev"])
    guesses.append(res)

# Calculate chi-square of the results, and choose the best one
# fitting.reinitialize moves the parameter vector away from edges of the prior.
chisq = [np.sum(r.fun**2) for r in guesses]
best = np.argmin(chisq)
theta_best = fitting.reinitialize(guesses[best].x, model,
                                  edge_trunc=run_params.get('edge_trunc', 0.1))
initial_prob = None
pdur = time.time() - ts

# output results
print('done {0} in {1}s'.format(min_method, pdur))
print('best {0} chi-sq: {1}'.format(min_method, chisq[best]))
print('best guess paramaters:')
for k, t in zip(model.theta_labels(), theta_best):
    print('  {} = {}'.format(k, t))
print pdur


[ 1.00000000e+08  0.00000000e+00  0.00000000e+00 -5.00000000e-01
  3.00000000e-01  4.00000000e+00  8.33333333e-01  8.00000000e-01
  7.50000000e-01  6.66666667e-01  5.00000000e-01  1.00000000e+00
  1.00000000e+00  0.00000000e+00  1.00000000e-02]


/Applications/anaconda/lib/python2.7/site-packages/prospect-0.2-py2.7.egg/prospect/models/priors.py:155: RuntimeWarning: divide by zero encountered in log


ValueError: Residuals are not finite in the initial point.

### View model after minimization
Now let's see how our model looks in the data space after minimization.

That should look much better, except maybe for the filters that we have masked out.

## Sampling the Posterior
Now that we're somewhat burned in, we can begin sampling from the posterior using **Markov Chain Monte Carlo** (MCMC). Prospector by default uses **emcee**, and will try to parallelize the process over multiple cores when available through MPI and mpi4py. In this interactive notebook though we will assume single-threaded operation. 

The `emcee` algorithm requires several options to be specified, related to the number of walkers, the number of iterations, and to rounds of burn-in.  For convenience we will store these in the `run_params` meta-parameter dictionary that we've been using.

In [ ]:
# Number of emcee walkers
run_params["nwalkers"] = 64
# Number of iterations of the MCMC sampling
run_params["niter"] = 512
# Number of iterations in each round of burn-in
# After each round, the walkers are reinitialized based on the 
# locations of the highest probablity half of the walkers.
run_params["nburn"] = [16, 32, 64]
# The following number controls how often the chain is written to disk. This can be useful 
# to make sure that not all is lost if the code dies during a long MCMC run. It ranges 
# from 0 to 1; the current chains will be written out every `interval` * `niter` iterations.
# The default is 1, i.e. only write out at the end of the run.
run_params["interval"] = 0.25 # write out after every 25% of the sampling is completed.

Let's go ahead and start sampling!

In [ ]:
# suppress output
fout = sys.stdout
fnull = open(os.devnull, 'w')
sys.stdout = fnull

# set the initial center of the ball of walkers to the best optimization result
initial_center = theta_best.copy()

# Start sampling
tstart = time.time()  # time it
out = fitting.run_emcee_sampler(lnprobfn, initial_center, model,
                                pool=None, hdf5=hfile, **run_params)
esampler, burn_loc0, burn_prob0 = out
edur = time.time() - tstart

sys.stdout = fout

print('done emcee in {0}s'.format(edur))


### Write Output
Now that everything's all set, let's save our results to disk.  These will be written to 2 or 3 files beginning with the value of `outroot`.

In [ ]:
write_results.write_hdf5(hfile, run_params, model, obs, 
                         esampler, guesses,
                         toptimize=pdur, tsample=edur,
                         sampling_initial_center=initial_center,
                         post_burnin_center=burn_loc0,
                         post_burnin_prob=burn_prob0)

# The code below can be used to write python pickles (like IDL save files) to disk.  
# These are not necessary, but can be convenient.
#write_results.write_pickles(run_params, model, obs, esampler, guesses,
#                            outroot=outroot, toptimize=pdur, tsample=edur,
#                            sampling_initial_center=initial_center,
#                            post_burnin_center=burn_loc0,
#                            post_burnin_prob=burn_prob0)

print('Finished')

In [ ]:
print edur

# Visualizing the Results

There are a few basic plotting tools available to do a quick check on the results available in *prospect.io.read_results* and *prospect.utils.plotting*. We'll hack a few of these together in *plot_utils* here in the demo folder to make them a bit more amenable to plotting in this notebook.  We'll also import some functions useful for reading the output files we made.

In [ ]:
from prospect.io.read_results import results_from, get_sps
from prospect.io.read_results import traceplot, subcorner
print(outroot) # This is the start of the filename where we saved the results

## Reading output files
Reading our results from our Pickle or HDF5 file is straightforward using the `results_from` method.  This returns a "results" dictionary, the `obs` dictionary of data to which the model was fit, and the `SedModel` object that was used in the fitting.

In [ ]:
# grab results (dictionary), the obs dictionary, and our corresponding models
# When using parameter files set `dangerous=True`
res, obs, mod = results_from("{}_mcmc.h5".format(outroot), dangerous=False)
# let's look at what's stored in the `res` dictionary
print(res.keys())

## Plotting parameter traces
To see how our MCMC samples look, we can examine a few traces (that is, the evolution of the parameter value with iteration in the MCMC chain.)  You can use these plots (and the chains more generally) to assess whether the MCMC has converged, or if you need to sample for more iterations.

In [ ]:
chosen = np.random.choice(run_params["nwalkers"], 
                          size=10, replace=False)
tracefig = traceplot(res, figsize=(20,10), chains=chosen)

## Making a corner plot
Our samples more generally can be shown using a corner/triangle plot.  The `subtriangle()` method below is a very thin wrapper on Dan Foreman-Mackey's **corner.py** code.  We'll overplot the MAP value as blue lines.

In [ ]:
# maximum a posteriori (of the locations visited by the MCMC sampler)
imax = np.argmax(res['lnprobability'])
i, j = np.unravel_index(imax, res['lnprobability'].shape)
theta_max = res['chain'][i, j, :].copy()

print('Optimization value: {}'.format(initial_center))
print('MAP value: {}'.format(theta_max))
cornerfig = subcorner(res, start=0, thin=5, truths=theta_max, 
                      fig=subplots(5,5,figsize=(27,27))[0])

## Look at SEDs and residuals
Finally, let's just take a look at a random model drawn from our chains, and at the highest posterior probability model in the chain.  In this notebook we already have the `sps` object instantiated, but in general we may have to regenerate it based on information stored in the output file using the `prospect.io.read_results.get_sps` method.

In [ ]:
# randomly chosen parameters from chain
randint = np.random.randint
nwalkers, niter = run_params['nwalkers'], run_params['niter']
theta = res['chain'][randint(nwalkers), randint(niter)]

# generate models
# sps = get_sps(res)  # this works if using parameter files
mspec, mphot, mextra = model.mean_model(theta, obs, sps=sps)
mspec_map, mphot_map, _ = model.mean_model(theta_max, obs, sps=sps)


#plot setting-----
theta = model.theta.copy()
theta[model.theta_index["mass"]] = 1e8
initial_spec, initial_phot, initial_mfrac = model.sed(theta, obs=obs, sps=sps)
# spec, phot, x = sps.get_spectrum(outwave=obs['wavelength'], filters=obs["filters"], **model.params)
title_text = ','.join(["{}={}".format(p, model.params[p][0]) for p in model.free_params])

a = 1.0 + model.params.get('zred', 0.0) # cosmological redshifting
# photometric effective wavelengths
wphot = obs["phot_wave"]
# spectroscopic wavelengths
if obs["wavelength"] is None:
    # *restframe* spectral wavelengths, since obs["wavelength"] is None
    wspec = sps.wavelengths
    wspec *= a #redshift them
else:
    wspec = obs["wavelength"]
    
xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
temp = np.interp(np.linspace(xmin,xmax,10000), wspec, initial_spec)
ymin, ymax = temp.min()*0.8, temp.max()/0.4


# Make plot of data and model
figure(figsize=(16,8))

loglog(wspec, mspec, label='Model spectrum (random draw)',
       lw=0.7, color='navy', alpha=0.7)
loglog(wspec, mspec_map, label='Model spectrum (MAP)',
       lw=0.7, color='green', alpha=0.7)
errorbar(wphot, mphot, label='Model photometry (random draw)',
         marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
         markerfacecolor='none', markeredgecolor='blue', 
         markeredgewidth=3)
errorbar(wphot, mphot_map, label='Model photometry (MAP)',
         marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
         markerfacecolor='none', markeredgecolor='green', 
         markeredgewidth=3)
errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
         label='Observed photometry', ecolor='red', 
         marker='o', markersize=10, ls='', lw=3, alpha=0.8, 
         markerfacecolor='none', markeredgecolor='red', 
         markeredgewidth=3)

# plot transmission curves
for f in obs['filters']:
    w, t = f.wavelength.copy(), f.transmission.copy()
    while t.max() > 1:
        t /= 10.
    t = 0.1*(ymax-ymin)*t + ymin
    loglog(w, t, lw=3, color='gray', alpha=0.7)

xlabel('Wavelength [A]')
ylabel('Flux Density [maggies]')
xlim([xmin, xmax])
ylim([ymin, ymax])
legend(loc='best', fontsize=20)
tight_layout()